In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pickle
from sklearn import metrics
import seaborn as sns
from tensorflow.python.tools import freeze_graph

%matplotlib inline
plt.style.use('ggplot')

In [ ]:
column_names = ['user-id','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
x = 'x-axis'
y = 'y-axis'
z = 'z-axis'

def read_data(file_name):
    data = pd.read_csv(file_name, header = None, names = column_names)
    return data

def normalizeBySD(feature):
    mu = np.mean(feature,axis = 0)
    sigma = np.std(feature, axis = 0)
    return (feature - mu)/sigma

In [ ]:
features = read_data('data/WISDM_ar_v1.1_raw.txt')
features.dropna(axis=0, how='any', inplace= True)
features[z] = features[z].str.replace(';','')
features[z] = features[z].astype(np.float64)
features.drop_duplicates(column_names, keep= 'first', inplace=True)

In [ ]:
features[x] = normalizeBySD(features[x])
features[y] = normalizeBySD(features[y])
features[z] = normalizeBySD(features[z])

In [ ]:
activity = ['Walking', 'Jogging',  'Sitting', 'Standing', 'Downstairs', 'Upstairs']
step = 15
num_rows = 90

def segmentDataset(featureList):
    segments = []
    labels = []
    for i in range(0, len(featureList) - num_rows, step):
        xs = featureList[x].values[i: i + num_rows]
        ys = featureList[y].values[i: i + num_rows]
        zs = featureList[z].values[i: i + num_rows]
        try:
            label = stats.mode(featureList["activity"][i: i + num_rows])[0][0]
            if label in activity:
                labels = np.append(labels,label)
                segments.append([xs, ys, zs])
            else:
                print(i, i+num_rows)
        except TypeError:
            print(i, i+num_rows)
            
    return segments, labels

In [ ]:
segments, new_label = segmentDataset(features)
labels = np.asarray(pd.get_dummies(new_label), dtype = np.int8)
segments = np.array(segments)
reshaped_segments = segments.reshape(len(segments),1, 90, 3)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(reshaped_segments, labels, test_size=0.3)

In [ ]:
input_height = 1
input_width = 90
num_labels = 6
batch_size = 10
input_name = 'input'
output_name = 'output'

kernel_size = 60
depth = 60
num_channels = 3

kernel_size_pool = 20
stride_pool = 2

kernel_size2 = 6
depth2 = 6
num_channel2 = depth * num_channels

num_hidden = 256

In [ ]:
def initialize_weights(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def initialize_biases(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

In [ ]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels], name=input_name)

conv1_filters = initialize_weights([1, kernel_size, num_channels, depth])
conv1_biases = initialize_biases([depth * num_channels])
c = tf.nn.relu(tf.add(tf.nn.depthwise_conv2d(X, conv1_filters, [1, 1, 1, 1], padding='VALID'), conv1_biases))

p = tf.nn.max_pool(c, ksize=[1, 1, kernel_size_pool, 1], strides=[1, 1, stride_pool, 1], padding='VALID')

conv2_filters = initialize_weights([1, kernel_size2, num_channel2, depth2])
conv2_biases = initialize_biases([depth2 * num_channel2])
c = tf.nn.relu(tf.add(tf.nn.depthwise_conv2d(p, conv2_filters, [1, 1, 1, 1], padding='VALID'), conv2_biases))

shape = c.get_shape().as_list()
c_flat = tf.reshape(c, [-1, shape[1] * shape[2] * shape[3]])

flat_weights = initialize_weights([shape[1] * shape[2] * depth * num_channels * (depth2), num_hidden])
flat_biases = initialize_biases([num_hidden])
flat = tf.nn.tanh(tf.add(tf.matmul(c_flat, flat_weights),flat_biases))

out_weights = initialize_weights([num_hidden, num_labels])
out_biases = initialize_biases([num_labels])
pred_Y = tf.matmul(flat, out_weights) + out_biases
y_ = tf.nn.softmax(pred_Y, name=output_name)

In [ ]:
learning_rate = 0.0001
training_epochs = 2 #50
total_batches = train_x.shape[0] // batch_size

Y = tf.placeholder(tf.float32, shape=[None,num_labels])
loss = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

right_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(right_prediction, tf.float32))

In [ ]:
saver = tf.train.Saver()
history = dict(train_loss=[], train_acc=[], test_loss=[], test_acc=[])

with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            session.run(optimizer,feed_dict={X: batch_x, Y : batch_y})
            
        _, loss_tr, accuracy_tr = session.run([y_, loss, accuracy],feed_dict={X: train_x, Y : train_y})
        _, loss_te, accuracy_te = session.run([y_, loss, accuracy], feed_dict={X: test_x, Y: test_y})
        history['train_loss'].append(loss_tr)
        history['train_acc'].append(accuracy_tr)
        history['test_loss'].append(loss_te)
        history['test_acc'].append(accuracy_te)
        
        
        print ("Epoch: ",epoch," Training Loss: ",loss_tr," Training Accuracy: ", accuracy_tr)
        
    predictions, acc_final, loss_final = session.run([y_, accuracy, loss], feed_dict={X: test_x, Y: test_y})
    print ("Final loss:", loss_final, "Final Accuracy:", acc_final)
    pickle.dump(predictions, open("predictions.p", "wb"))
    pickle.dump(history, open("history.p", "wb"))
    tf.train.write_graph(session.graph_def, '.', './checkpoint/har.pbtxt')  
    saver.save(session, save_path = "./checkpoint/har.ckpt")

In [ ]:
history = pickle.load(open("history.p", "rb"))
predictions = pickle.load(open("predictions.p", "rb"))

In [ ]:
fig, ax1 = plt.subplots(figsize=(18, 12))

fig.suptitle("Training session's progress over iterations", fontsize=20)
ax2 = ax1.twinx()

ax2.plot(np.array(history['train_loss']), "r--", label="Train loss")
ax2.plot(np.array(history['test_loss']), "r-", label="Test loss")

ax1.plot(np.array(history['train_acc']), "g--", label="Train accuracy")
ax1.plot(np.array(history['test_acc']), "g-", label="Test accuracy")

ax1.legend(shadow=True, loc='best', fontsize=20)
ax2.legend(shadow=True, loc='right', fontsize=20)

ax2.set_ylabel('Loss values', fontsize=20)
ax1.set_xlabel('Training Epoch', fontsize=20)
ax1.set_ylabel('Accuracy values', fontsize=20)

plt.show()

In [ ]:
LABELS = ['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking']
max_test = np.argmax(test_y, axis=1)
max_predictions = np.argmax(predictions, axis=1)
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions)
plt.figure(figsize=(16, 14))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d", cmap="YlGnBu");
plt.title("Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
MODEL_NAME = 'har'

input_graph_path = 'checkpoint/' + MODEL_NAME+'.pbtxt'
checkpoint_path = './checkpoint/' +MODEL_NAME+'.ckpt'
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = 'har_classifier.pb'

freeze_graph.freeze_graph(input_graph_path, input_saver="",
                          input_binary=False, input_checkpoint=checkpoint_path, 
                          output_node_names=output_name, restore_op_name="save/restore_all",
                          filename_tensor_name="save/Const:0", 
                          output_graph=output_frozen_graph_name, clear_devices=True, initializer_nodes="")